In [ ]:
from prospect.area import Area
from prospect.coverage import Coverage
%matplotlib inline
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
shp = gpd.GeoDataFrame.from_file(filename='../data/area_test/Survey_Fields_Master.shp')
areas = []
for i in range(0, 20):
    temp_area = Area.from_shapely_polygon(name=f'{shp.MASA[i]}{shp.PARCELA[i]}{shp.SUBPARCE[i]}', polygon=shp.geometry[i])
    areas.append(temp_area)

In [ ]:
for area in areas:
    opt_or_long = Coverage.make_transects(area, 
                                        name='test_transects', 
                                        spacing = 10, 
                                        sweep_width=2, 
                                        orientation=40, 
                                        optimize_orient_by='area_orient', 
                                        orient_axis='long', 
                                        min_time_per_unit=1)
    opt_or_long.df.plot(ax=area.df.plot(), color='yellow')

In [ ]:
for area in areas:
    opt_or_short = Coverage.make_transects(area, 
                                        name='test_transects', 
                                        spacing = 10, 
                                        sweep_width=2, 
                                        orientation=40, 
                                        optimize_orient_by='area_orient', 
                                        orient_axis='short', 
                                        min_time_per_unit=1)
    opt_or_short.df.plot(ax=area.df.plot(), color='yellow');

In [ ]:
for area in areas:
    opt_or_long_radials = Coverage.make_radials(area,
                                    name='test_radials',
                                    spacing=10,
                                    radius=2,
                                    orientation=40,
                                    optimize_orient_by='area_orient',
                                    orient_axis='long',
                                    min_time_per_unit=1
                                   )
    opt_or_long_radials.df.plot(ax=area.df.plot(), color='yellow')

In [ ]:
for area in areas:
    opt_or_short_radials = Coverage.make_radials(area,
                                    name='test_radials',
                                    spacing=10,
                                    radius=2,
                                    orientation=40,
                                    optimize_orient_by='area_orient',
                                    orient_axis='short',
                                    min_time_per_unit=1
                                   )
    opt_or_short_radials.df.plot(ax=area.df.plot(), color='yellow')

Find the minimum rotated rectangle
Find point with min y
Find point with max x
Calculate distance from Point with min y to Point with max x
Calculate angle from Point with min y to Point with max x


In [ ]:
min(np.asarray(area.df.geometry[0].minimum_rotated_rectangle.exterior.coords).T[0])

In [ ]:
import math
coords = pd.DataFrame(np.array(area.df.geometry[0].minimum_rotated_rectangle.exterior.coords), columns=['x', 'y'])
pt_ymin = list(coords.loc[coords['y']==coords['y'].min()].itertuples(index=False, name=None))[0]
pt_xmax = list(coords.loc[coords['x']==coords['x'].max()].itertuples(index=False, name=None))[0]
pt_xmin = list(coords.loc[coords['x']==coords['x'].min()].itertuples(index=False, name=None))[0]

opp = pt_xmax[1]-pt_ymin[1]  # sides of the "triangle"
adj = pt_xmax[0]-pt_ymin[0]

right_side = math.sqrt((adj)**2 + (opp)**2)
left_side = math.sqrt((pt_ymin[0]-pt_xmin[0])**2 + (pt_ymin[1]-pt_xmin[1])**2)

In [ ]:
right_side, left_side

In [ ]:
for xy in list(area.df.geometry[0].minimum_rotated_rectangle.exterior.coords):
    

In [ ]:
min_rotated_rect = area.df.geometry[0].minimum_rotated_rectangle
min_rot_angle = area.df.geometry[0].minimum_rotated_rectangle
list(min_rotated_rect.bounds)
c = min_rotated_rect.centroid
type(c)

In [ ]:
import numpy as np

In [ ]:
a = np.array(np.meshgrid(np.arange(1, 5), np.arange(1, 5))).T.reshape(-1, 2)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
gpd.GeoSeries([Point(xy) for xy in a]).plot()

In [ ]:
from math import sqrt
from shapely.geometry import LineString, Point
# find longest dimension (longest diagonal of bbox)
bounds = area.df.bounds
width = bounds.maxx.max() - bounds.minx.min()
height = bounds.maxy.max() - bounds.miny.max()
diag_dist = sqrt(width**2 + height**2)

# calculate num of lines
n_transects = int(diag_dist // spacing)

# start at centroid of bounding box
centroid = area.df.boundary.centroid.iloc[0]

# fill out lines
# calculate x values
if n_transects % 2 == 0:  # even num transects
    left_start = centroid.x - spacing/2
    right_start = centroid.x + spacing/2
    left_xs = left_start - (np.arange(0, n_transects/2) * spacing)
    right_xs = right_start + (np.arange(0, n_transects/2) * spacing)
    xs = np.sort(np.concatenate([left_xs, right_xs]))
else:  # odd num transects
    start_x = centroid.x
    left_xs = start_x - (np.arange(1, n_transects/2) * spacing)
    right_xs = start_x + (np.arange(1, n_transects/2) * spacing)
    xs = np.sort(np.insert(np.concatenate([left_xs, right_xs]), 1, start_x))
    
y_max = centroid.y + diag_dist / 2
y_min = centroid.y - diag_dist / 2

top_coords = list(zip(xs, np.full_like(xs, fill_value=y_max)))
bottom_coords = list(zip(xs, np.full_like(xs, fill_value=y_min)))

lines_gs = gpd.GeoSeries([LineString(coord_pair) for coord_pair in zip(top_coords, bottom_coords)])
lines_gs = lines_gs.rotate(angle, origin = Point(centroid.x, centroid.y))
lines_gdf = gpd.GeoDataFrame({'geometry': lines_gs}, 
                             geometry='geometry')

# clip lines by area
poly = area.df.geometry.unary_union
spatial_index = lines_gdf.sindex
bbox = poly.bounds
sidx = list(spatial_index.intersection(bbox))
lines_sub = lines_gdf.iloc[sidx]
clipped = lines_sub.copy()
clipped['geometry'] = lines_sub.intersection(poly)
lines_clipped = clipped[clipped.geometry.notnull()]


transects_buffer = lines_clipped.buffer(sweep_width)  # buffer transects
buffer_gdf = gpd.GeoDataFrame({'angle_deg':[angle] * transects_buffer.shape[0],
                               'length': lines_clipped.length,
                               'geometry': transects_buffer}, 
                               geometry='geometry')

transects = gpd.overlay(buffer_gdf, area.df, how='intersection')
transects['area'] = transects.area
transects['min_search_time'] = sec_per_unit_dist * transects['length']
# add su_id column
transects['su_id'] = [i for i in range(transects.shape[0])]
transects = transects.loc[:, ['su_id', 'angle_deg', 'length', 'area_name', 'visibility', 'geometry', 'area', 'min_search_time']]

transects.plot()

In [ ]:
transects

In [ ]:
centroid.x, spacing/2, left_xs, right_xs

In [ ]:
lines_gdf.plot(ax=area.df.plot(), color='red')

In [ ]:
from shapely.geometry import Point
lines_gs.rotate(110, origin = Point(centroid.x, centroid.y)).plot(ax=area.df.plot(), color='red')